<a href="https://colab.research.google.com/github/mbribiescaucla-afk/imperial-valley-et/blob/main/ET_AA_O_DifModels.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# --- 1. INSTALACIÓN Y AUTENTICACIÓN ---
!pip install -q earthengine-api geopandas

import geopandas as gpd
import pandas as pd
import ee
import json
import os
import time
from google.colab import drive

drive.mount('/content/drive')
ee.Authenticate()
ee.Initialize(project='ee-mbribiescaucla')

# --- 2. CARGAR SHAPEFILE ---
shapefile_path = '/content/drive/MyDrive/Shapefiles/CSB_IV_08_15.shp'
gdf = gpd.read_file(shapefile_path)

for y in range(2008, 2016):
    gdf[f'CDL{y}'] = pd.to_numeric(gdf[f'CDL{y}'], errors='coerce')

# --- 3. IDENTIFICAR PATRONES DE CAMBIO DE ALFALFA A OTROS CULTIVOS ---
pattern_results = []

for n_initial36 in range(7, 0, -1):  # Años iniciales con alfalfa
    n_final_non36 = 8 - n_initial36  # Años finales ≠ alfalfa
    initial_years = [f'CDL{2008 + i}' for i in range(n_initial36)]
    final_years = [f'CDL{2008 + i}' for i in range(n_initial36, 8)]
    pattern_label = 'A' * n_initial36 + 'O' * n_final_non36
    required_cols = initial_years + final_years
    gdf_clean = gdf.dropna(subset=required_cols)

    def match_pattern(row):
        if any(row[y] != 36 for y in initial_years): return False
        if any(row[y] == 36 for y in final_years): return False
        return True

    matched = gdf_clean[gdf_clean.apply(match_pattern, axis=1)].copy()
    matched['pattern'] = pattern_label
    pattern_results.append(matched)

# Concatenar resultados
gdf_all = pd.concat(pattern_results)
print("Total de polígonos seleccionados:", len(gdf_all))


Mounted at /content/drive
Total de polígonos seleccionados: 1937


In [ ]:
# --- 4. EXPORTAR GEOJSON A EE ---
gdf_all = gdf_all.to_crs("EPSG:4326")
geojson_path = '/content/all_patterns_from_alfalfa.json'
if os.path.exists(geojson_path): os.remove(geojson_path)
gdf_all.to_file(geojson_path, driver='GeoJSON')

with open(geojson_path) as f:
    geojson_data = json.load(f)

features = [ee.Feature(ee.Geometry(f['geometry']), f['properties']) for f in geojson_data['features']]
fc = ee.FeatureCollection(features)
print("Polígonos cargados en EE:", fc.size().getInfo())


Polígonos cargados en EE: 1937


In [ ]:
# --- 5. CALCULAR ET DIARIA PROMEDIO PARA ENSEMBLE, DISALEXI Y EEMETRIC ---
results_list = []

modelos = ['ensemble', 'disalexi', 'eemetric']
colecciones = {
    'ensemble': ('OpenET/ENSEMBLE/CONUS/GRIDMET/MONTHLY/v2_0', 'et_ensemble_sam'),
    'disalexi': ('OpenET/DISALEXI/CONUS/GRIDMET/MONTHLY/v2_0', 'et'),
    'eemetric': ('OpenET/EEMETRIC/CONUS/GRIDMET/MONTHLY/v2_0', 'et')
}

for year in range(2008, 2016):
    print(f"🔄 Año {year}...")

    start = ee.Date.fromYMD(year, 1, 1)
    end = ee.Date.fromYMD(year, 12, 31)

    for model in modelos:
        print(f"  ➤ Modelo: {model.upper()}")
        collection_id, band_name = colecciones[model]
        col = ee.ImageCollection(collection_id).filterDate(start, end)
        image = col.select(band_name).sum()

        reduced = image.reduceRegions(
            collection=fc,
            reducer=ee.Reducer.mean(),
            scale=30
        ).map(lambda f: f.set('year', year)
                      .set('model', model.upper())
                      .set('et_mm_day', ee.Number(f.get('mean')).divide(365)))

        results_list.append(reduced)

all_results = ee.FeatureCollection(results_list).flatten()


🔄 Año 2008...
  ➤ Modelo: ENSEMBLE
  ➤ Modelo: DISALEXI
  ➤ Modelo: EEMETRIC
🔄 Año 2009...
  ➤ Modelo: ENSEMBLE
  ➤ Modelo: DISALEXI
  ➤ Modelo: EEMETRIC
🔄 Año 2010...
  ➤ Modelo: ENSEMBLE
  ➤ Modelo: DISALEXI
  ➤ Modelo: EEMETRIC
🔄 Año 2011...
  ➤ Modelo: ENSEMBLE
  ➤ Modelo: DISALEXI
  ➤ Modelo: EEMETRIC
🔄 Año 2012...
  ➤ Modelo: ENSEMBLE
  ➤ Modelo: DISALEXI
  ➤ Modelo: EEMETRIC
🔄 Año 2013...
  ➤ Modelo: ENSEMBLE
  ➤ Modelo: DISALEXI
  ➤ Modelo: EEMETRIC
🔄 Año 2014...
  ➤ Modelo: ENSEMBLE
  ➤ Modelo: DISALEXI
  ➤ Modelo: EEMETRIC
🔄 Año 2015...
  ➤ Modelo: ENSEMBLE
  ➤ Modelo: DISALEXI
  ➤ Modelo: EEMETRIC


In [ ]:
# --- 6. EXPORTAR A DRIVE ---
export_desc = 'ET_daily_Ensemble_DisAlexi_eeMETRIC_fromAlfalfa_2008_2015'
task = ee.batch.Export.table.toDrive(
    collection=all_results,
    description=export_desc,
    folder='Colab_ET',
    fileFormat='CSV'
)
task.start()

while task.active():
    print("Esperando exportación...")
    time.sleep(10)

print("✅ Exportación completada:", task.status())


Esperando exportación...
Esperando exportación...
Esperando exportación...
Esperando exportación...
Esperando exportación...
Esperando exportación...
Esperando exportación...
✅ Exportación completada: {'state': 'COMPLETED', 'description': 'ET_daily_Ensemble_DisAlexi_eeMETRIC_fromAlfalfa_2008_2015', 'priority': 100, 'creation_timestamp_ms': 1750799909660, 'update_timestamp_ms': 1750799980116, 'start_timestamp_ms': 1750799912874, 'task_type': 'EXPORT_FEATURES', 'destination_uris': ['https://drive.google.com/#folders/1gwo7ys-IUcg4JYI4ytcC5_hgJRJdymR3'], 'attempt': 1, 'batch_eecu_usage_seconds': 1005.046875, 'id': '7IEM465C46V7SZ5JJFHGYMY6', 'name': 'projects/ee-mbribiescaucla/operations/7IEM465C46V7SZ5JJFHGYMY6'}


In [ ]:
# --- 7. PROCESAR CSV Y EXPORTAR A EXCEL ---
import pandas as pd

csv_path = '/content/drive/MyDrive/Colab_ET/ET_daily_Ensemble_DisAlexi_eeMETRIC_fromAlfalfa_2008_2015.csv'
df = pd.read_csv(csv_path)

required_cols = ['CSBID', 'year', 'model', 'et_mm_day']
for col in required_cols:
    if col not in df.columns:
        raise ValueError(f"Falta la columna requerida: {col}")

df_pivot = df.pivot_table(
    index='CSBID',
    columns=['model', 'year'],
    values='et_mm_day'
).reset_index()

df_pivot.columns = ['CSBID'] + [f"ET {model} {year}" for model, year in df_pivot.columns.tolist()[1:]]

cdl_cols = [f'CDL{y}' for y in range(2008, 2016)]
base_cols = ['CSBID', 'AreaRec', 'pattern'] + cdl_cols
df_base = df[base_cols].drop_duplicates(subset='CSBID')

df_final = pd.merge(df_base, df_pivot, on='CSBID', how='left')
et_cols = sorted([col for col in df_final.columns if col.startswith('ET ')])
df_final = df_final[['CSBID', 'AreaRec', 'pattern'] + cdl_cols + et_cols]

output_excel = '/content/drive/MyDrive/Colab_ET/ET_Patrones_from_Alfalfa_2008_2015.xlsx'
df_final.to_excel(output_excel, index=False)

print("✅ Excel generado:", output_excel)


✅ Excel generado: /content/drive/MyDrive/Colab_ET/ET_Patrones_from_Alfalfa_2008_2015.xlsx
